In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump

# Step 1: Load the dataset (Assuming the labeled flow data is in CSV format)
data = pd.read_csv('cleaned_data.csv')

# Step 2: Inspect and Preprocess Data (Dropping unnecessary columns, handling missing values, etc.)
data = data.drop(columns=['Timestamp'], errors='ignore')  # Modify based on dataset structure

# Step 3: Separate features (X) and target labels (y)
X = data.drop('label', axis=1)  # Features
y = data['label']  # Target (Normal or Attack)

# Step 4: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 5: Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Random Forest Model Accuracy: {accuracy * 100:.2f}%')

# Step 7: Save the trained model for later use
dump(rf_model, 'random_forest_iot23_model.joblib')

# Step 8: Save the processed data into a CSV file for further analysis or reproducibility
X_train.to_csv('iot23_processed_train_data.csv', index=False)
X_test.to_csv('iot23_processed_test_data.csv', index=False)


Random Forest Model Accuracy: 81.43%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.download('random_forest_iot23_model.joblib')
files.download('iot23_processed_train_data.csv')
files.download('iot23_processed_test_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# After Step 6: Evaluate the model
y_pred = rf_model.predict(X_test)

# Create a DataFrame for results
results_df = X_test.copy()  # Copy the test features
results_df['True Label'] = y_test.reset_index(drop=True)  # True labels
results_df['Predicted Label'] = y_pred  # Predictions

# Save the results to a CSV file
results_file_path = 'iot23_results.csv'
results_df.to_csv(results_file_path, index=False)

# Optional: Print some results to check
print(results_df.head())


       duration  orig_bytes  resp_bytes  missed_bytes  orig_pkts  \
42995  3.155947           0           0           0.0        3.0   
17812  2.999841           0           0           0.0        3.0   
12377  2.997684           0           0           0.0        3.0   
44381  0.000465          29          45           0.0        1.0   
19241  2.996074           0           0           0.0        3.0   

       orig_ip_bytes  resp_pkts  resp_ip_bytes  proto_icmp  proto_tcp  ...  \
42995          180.0        0.0            0.0           0          1  ...   
17812          180.0        0.0            0.0           0          1  ...   
12377          180.0        0.0            0.0           0          1  ...   
44381           57.0        1.0           73.0           0          0  ...   
19241          180.0        0.0            0.0           0          1  ...   

       conn_state_RSTRH  conn_state_S0  conn_state_S1  conn_state_S2  \
42995                 0              1            

In [ ]:
from google.colab import files
files.download('iot23_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump

# Step 1: Load the dataset
data = pd.read_csv('iot23_combined.csv')  # Ensure this is the correct path to your dataset

# Step 2: Drop unnecessary columns if any
data = data.drop(columns=['Timestamp'], errors='ignore')  # Modify based on dataset structure

# Separate features (X) and target labels (y)
X = data.drop('label', axis=1)  # Features
y = data['label']  # Target (Normal or Attack)

# Step 3: Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Step 4: Create a Column Transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', X.select_dtypes(include=['int64', 'float64']).columns),  # Numeric features pass through
        ('cat', OneHotEncoder(), categorical_cols)  # Apply One-Hot Encoding to categorical features
    ])

# Step 5: Create a Pipeline that first transforms the data then fits the model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Step 6: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 7: Train the model
pipeline.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Random Forest Model Accuracy: {accuracy * 100:.2f}%')

# Step 9: Create a DataFrame for results with only important columns
important_columns = [
    'ts',
    'id.orig_h',
    'duration',
    'orig_bytes',
    'resp_bytes',
    'missed_bytes',
    'orig_pkts',
    'orig_ip_bytes',
    'resp_pkts',
    'resp_ip_bytes',
    'label',
    'proto_icmp',
    'proto_tcp',
    'proto_udp',
    'conn_state_SF',
    'conn_state_REJ',
    'conn_state_RSTO',
    'conn_state_S1',
    'conn_state_S0'
]

# Create the results DataFrame
results_df = pd.DataFrame({
    'True Label': y_test.reset_index(drop=True),
    'Predicted Label': y_pred
})

# Include the important features from X_test
results_df = pd.concat([X_test[important_columns].reset_index(drop=True), results_df], axis=1)

# Step 10: Save the results to a CSV file
results_file_path = 'important_results.csv'
results_df.to_csv(results_file_path, index=False)

# Step 11: Save the trained model for later use
dump(pipeline, 'random_forest_iot23_model.joblib')

# Optional: Print some results to check
print(results_df.head())


ValueError: Found unknown categories ['223.32.52.214', '89.14.63.89', '45.202.33.75', '213.219.200.149', '64.111.83.105', '192.72.62.73', '24.165.148.231', '69.163.134.27', '200.56.101.169', '61.125.141.34', '38.64.137.227', '121.41.16.177', '191.130.209.104', '218.248.160.166', '173.244.96.250', '66.165.163.178', '45.112.246.17', '200.2.162.122', '193.200.241.216', '203.80.217.46', '82.232.137.29', '181.24.133.99', '37.236.121.123', '149.172.45.234', '203.249.77.1', '174.144.241.185', '178.192.31.3', '58.160.249.10', '42.56.109.169', '128.109.9.53', '130.81.21.240', '161.246.231.2', '216.218.222.230', '182.163.62.19', '101.29.12.13', '103.70.108.1', '211.175.185.110', '195.1.208.169', '213.131.110.26', '104.203.116.205', '115.2.199.242', '112.190.237.158', '190.19.196.169', '173.151.74.200', '37.158.210.116', '182.72.26.5', '67.129.123.118', '1.208.92.138', '198.202.27.3', '141.101.178.40', '78.110.192.158', '36.236.162.86', '200.24.33.150', '126.178.165.105', '213.248.86.150', '103.2

In [ ]:
with open("/content/drive/MyDrive/dataset iot/CTU-IoT-Malware-Capture-1-1/bro/conn.log.labeled", "r") as file:
    for i in range(10):  # Adjust number of lines as needed
        print(f"Line {i+1}: {file.readline()}")

Line 1: #separator \x09

Line 2: #set_separator	,

Line 3: #empty_field	(empty)

Line 4: #unset_field	-

Line 5: #path	conn

Line 6: #open	2018-05-21-21-03-43

Line 7: #fields	ts	uid	id.orig_h	id.orig_p	id.resp_h	id.resp_p	proto	service	duration	orig_bytes	resp_bytes	conn_state	local_orig	local_resp	missed_bytes	history	orig_pkts	orig_ip_bytes	resp_pkts	resp_ip_bytes	tunnel_parents   label   detailed-label

Line 8: #types	time	string	addr	port	addr	port	enum	string	interval	count	count	string	bool	bool	count	string	count	count	count	count	set[string]   string   string

Line 9: 1525879831.015811	CUmrqr4svHuSXJy5z7	192.168.100.103	51524	65.127.233.163	23	tcp	-	2.999051	0	0	S0	-	-	0	S	3	180	0	0	(empty)   Malicious   PartOfAHorizontalPortScan

Line 10: 1525879831.025055	CH98aB3s1kJeq6SFOc	192.168.100.103	56305	63.150.16.171	23	tcp	-	-	-	-	S0	-	-	0	S	1	60	0	0	(empty)   Malicious   PartOfAHorizontalPortScan



In [ ]:
import pandas as pd

# Read the labeled conn.log file into a DataFrame
file_path = "/content/drive/MyDrive/dataset iot/CTU-IoT-Malware-Capture-1-1/bro/conn.log.labeled"
df = pd.read_csv(file_path, sep="\t", skiprows=6)

# Print the first few rows of the DataFrame
print(df.head())

# Access specific columns
print(df["id.orig_h"])  # Access the 'id.orig_h' column

# Convert 'duration' column to numeric, coercing errors to NaN
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')

# Drop rows with NaN in the 'duration' column
df = df.dropna(subset=['duration'])

# Filter rows based on conditions
filtered_df = df[df['duration'] > 10]  # Connections longer than 10 seconds

# Group and aggregate data
grouped_df = df.groupby("id.orig_h").agg({"duration": "mean"})  # Calculate average duration per source IP

# Save the filtered data to a CSV file
filtered_df.to_csv("/content/filtered_data.csv", index=False)

# Save the grouped data to a CSV file
grouped_df.to_csv("/content/grouped_data.csv")

print("Data saved successfully!")

<ipython-input-12-cb7010da2a39>:5: DtypeWarning: Columns (0,3,5,14,16,17,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep="\t", skiprows=6)


             #fields                  ts              uid id.orig_h  \
0             #types                time           string      addr   
1  1525879831.015811  CUmrqr4svHuSXJy5z7  192.168.100.103     51524   
2  1525879831.025055  CH98aB3s1kJeq6SFOc  192.168.100.103     56305   
3  1525879831.045045   C3GBTkINvXNjVGtN5  192.168.100.103     41101   
4  1525879832.016240   CDe43c1PtgynajGI6  192.168.100.103     60905   

         id.orig_p id.resp_h id.resp_p proto   service  duration  ...  \
0             port      addr      port  enum    string  interval  ...   
1   65.127.233.163        23       tcp     -  2.999051         0  ...   
2    63.150.16.171        23       tcp     -         -         -  ...   
3     111.40.23.49        23       tcp     -         -         -  ...   
4  131.174.215.147        23       tcp     -  2.998796         0  ...   

  conn_state local_orig local_resp missed_bytes history orig_pkts  \
0     string       bool       bool        count  string     count